<h1>SJ Training fNIRS Data Analysis</h1>
Written by Ansley Kunnath
Updated July 29, 2024
Python v3.9.12

In [1]:
# Import packages
import PyQt5
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import warnings
import pooch
import pyarrow
import pandas as pd
import glob
import csv
#import pyvistaqt
#import pyvista
from os import path as op
from itertools import compress
from collections import defaultdict
from scipy import signal, stats

import statsmodels.formula.api as smf
import mne_nirs.preprocessing
import mne_nirs.statistics
import mne_nirs.utils
import mne_nirs.statistics
import mne
from mne.viz import plot_compare_evokeds
from mne.preprocessing.nirs import tddr
from nilearn.glm.first_level import make_first_level_design_matrix, compute_regressor  

# Open plots in new window
%matplotlib qt 

#warnings.filterwarnings("ignore", category=DeprecationWarning) 
#warnings.filterwarnings("ignore", category=FutureWarning) 

print("All done!") 

All done!


In [ ]:
# Locate files

subjects = ('201 202 203 204 205 206 207 208 209 212 213 214 215 216 217 218 219 221 223 224 225 226 228 229 230 231 234').split() #
#subjects = ('201 205 207 208 209 212 213 215 216 217 218 219 224').split() #

# Mapping of subjects to groups
subject_to_group = {
    201: "trained",
    202: "control",
    203: "trained",
    204: "control",
    205: "control",
    206: "control",
    207: "trained",
    208: "control",
    209: "control",
    212: "trained",
    213: "trained",
    214: "trained",
    215: "control",
    216: "trained",
    217: "control",
    218: "control",
    219: "trained",
#    220: "control", #sampling rate = 2.40 Hz instead of 4.8
    221: "trained",
    223: "trained",
    224: "control",
    225: "trained",
    226: "control",
    228: "trained",
    229: "control",
    230: "trained",
    231: "trained",
    234: "control",
}

sfreq = 4.807692
conditions = ('A', 'V', 'AV', 'W')
groups = ('trained','control')
days = ('1', '3')
runs = (1, 2)

condition_colors = dict(  # https://personal.sron.nl/~pault/data/colourschemes.pdf
    A='#4477AA',  # sblue
    AV='#CCBB44',  # yellow
    V='#EE7733',  # orange
    W='#AA3377',  # purple
)
exp_name = 'av'
duration = 1.8
design = 'event'
plot_subject = '201'
plot_day = 1
plot_run = 1
beh_title, beh_idx = 'AV', 0
filt_kwargs = dict(
    l_freq=0.02, l_trans_bandwidth=0.02,
    h_freq=0.2, h_trans_bandwidth=0.02)
run_h = True  # regenerate HbO/HbR
n_jobs = 8  # for GLM

# SET FOLDER LOCATIONS
# I save the output files outside of the folder that's uploaded to Github
current_directory = %pwd
os.chdir(current_directory)

raw_path = '../../data'
proc_path = '../../processed'
results_path = '../../results'
subjects_dir = '../../subjects'
os.makedirs(proc_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(subjects_dir, exist_ok=True)
#mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir, verbose=True)

use = None
all_sci = list()
plt.rcParams['axes.titlesize'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
got_bad = 0
got_total = 0

# Prep making bad channels report
bad_channels_filename = op.join(results_path, 'bad_channels_report.csv')
if not op.isfile(bad_channels_filename):
    with open(bad_channels_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Subject', 'Day', 'Run', 'Percent Bad'])

def normalize_channel_names(channels_set):
    return {name.split()[0] for name in channels_set}

def add_bad_channel_entry(subject, day, run, percentage_bad):
    with open(bad_channels_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([subject, day, run, f'{percentage_bad:.2f}%'])

# Sanity check for subjects
subjects_check = {int(subject) for subject in subjects}
subject_to_group_check = set(subject_to_group.keys())
if subjects_check == subject_to_group_check:
    print("All done!") 
    del subjects_check
    del subject_to_group_check
else:
    print("Error loading subject info") 

All done!


27


In [ ]:
###############################################################################
# Load participant data

#subjects = ('201 202').split() #for testing

for subject in subjects:
    for day in days:
        for run in runs:
            group = subject_to_group.get(int(subject), "unknown")
            root1 = f'Day{day}'
            root2 = f'{subject}_{day}'
            root3 = f'*-*-*_{run:03d}'
            fname_base = op.join(raw_path, root1, root2, root3)
            fname = glob.glob(fname_base)
            base = f'{subject}_{day}_{run:03d}'
            base_pr = base.ljust(20)
            # Save the plot subject
            if (op.isfile(op.join(proc_path, f'{base}_hbo_raw.fif')) and subject == plot_subject and run == plot_run):
                fname = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
                use = mne.io.read_raw_fif(fname)
                events, _ = mne.events_from_annotations(use)
                ch_names = [ch_name.rstrip(' hbo') for ch_name in use.ch_names]
                info = use.info
                print("Saved plot file!")
            # Don't resave old subjects...
            if not op.isfile(op.join(proc_path, f'{base}_hbo_raw.fif')):
                raw_intensity = mne.io.read_raw_nirx(fname[0])
                raw_od = mne.preprocessing.nirs.optical_density(
                    raw_intensity, verbose='error')
                # identify bad channels
                peaks = np.ptp(raw_od.get_data('fnirs'), axis=-1)
                flat_names = [
                    raw_od.ch_names[f].split(' ')[0]
                    for f in np.where(peaks < 0.001)[0]]
                sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
                all_sci.extend(sci)
                sci_mask = (sci < 0.25)
                got = np.where(sci_mask)[0]
                percentage_bad = (len(got) / len(raw_od.ch_names)) * 100
                print(f'    Run {base_pr}')
                # assign bads
                assert raw_od.info['bads'] == []
                bads = set(raw_od.ch_names[pick] for pick in got)
                bads = bads | set(ch_name for ch_name in raw_od.ch_names
                                if ch_name.split(' ')[0] in flat_names)
                bads = sorted(bads)
    #            raw_tddr = tddr(raw_od) # DON'T TDDR
    #            raw_tddr_bp = raw_tddr.copy()
    #            raw_tddr_bp = raw_tddr.copy().filter(**filt_kwargs) # DON'T BANDPASS FILTER
                raw_tddr_bp = raw_od.copy() # new
                raw_tddr_bp.info['bads'] = bads
                picks = mne.pick_types(raw_tddr_bp.info, fnirs=True)
                peaks = np.ptp(raw_tddr_bp.get_data(picks), axis=-1)
                assert (peaks > 1e-5).all()
                raw_tddr_bp.info['bads'] = [] 
                raw_h = mne.preprocessing.nirs.beer_lambert_law(raw_tddr_bp, 6.)
                h_bads = [
                    ch_name for ch_name in raw_h.ch_names
                    if ch_name.split(' ')[0] in set(bad.split(' ')[0] for bad in bads)]
                set_bads = set(bads) 
                set_h_bads = set(h_bads)
                normalized_bads = normalize_channel_names(set_bads)
                normalized_h_bads = normalize_channel_names(set_h_bads)
                assert normalized_bads == normalized_h_bads
    #            assert len(bads) == len(h_bads)
                raw_h.info['bads'] = h_bads
                raw_h.info._check_consistency()
                picks = mne.pick_types(raw_h.info, fnirs=True)
                peaks = np.ptp(raw_h.get_data(picks), axis=-1)
                assert (peaks > 1e-9).all()  
                # Interpolate bad channels (flat or low SCI)
                raw_h_interp = raw_h.copy().interpolate_bads(reset_bads=True, method = dict(fnirs = 'nearest')) # TEST
                raw_h_interp.save(op.join(proc_path, f'{base}_hbo_raw.fif'), 
                        overwrite=True)  
                assert len(raw_h.ch_names) == len(raw_h_interp.ch_names)
                # Save the bad channel data
                add_bad_channel_entry(subject, day, run, percentage_bad)
                del raw_intensity, raw_od, raw_tddr_bp, raw_h #, raw_tddr
                del normalized_h_bads, normalized_bads, set_bads, set_h_bads

print("All done!") 

In [ ]:
# Remove patients with over 25% bad channels on average across days and runs

subjects_to_remove = ['202', '203', '204', '206', '214', '221', '223', '226']

# Initialize counters for each group
removed_trained = 0
removed_control = 0

# Count and remove the subjects
for subject in subjects_to_remove:
    subject_int = int(subject)  # Convert to integer for dictionary key comparison
    if subject_int in subject_to_group:
        # Increment the appropriate counter based on the group of the subject
        if subject_to_group[subject_int] == "trained":
            removed_trained += 1
        elif subject_to_group[subject_int] == "control":
            removed_control += 1
        # Remove the subject from the dictionary
        subject_to_group.pop(subject_int, None)

# Update the subjects list after counting the removed subjects
subjects = [subject for subject in subjects if subject not in subjects_to_remove]

# Output the results
print(f'Removed {removed_trained} trained subjects.')
print(f'Removed {removed_control} control subjects.')

In [ ]:
###############################################################################
# Channel example figure
sfreq = 4.807692

for subject in subjects:
    for day in days:
        for run in runs:
            fname = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname)
            events, _ = mne.events_from_annotations(raw_h)
            #print(len(events))

def _make_design(raw_h, design, subject=None, run=None, day=None, group=None):
    annotations_to_remove = raw_h.annotations.description == '255.0'
    raw_h.annotations.delete(annotations_to_remove)
    events, _ = mne.events_from_annotations(raw_h)
    rows_to_remove = events[:, -1] == 1
    events = events[~rows_to_remove]
    # mis-codings
    if len(events)==101:
        events = events[1:]
    n_times = len(raw_h.times)
    stim = np.zeros((n_times, 4))
    events[:, 2] -= 1
    assert len(events) == 100, len(events)
    want = [0] + [25] * 4
    count = np.bincount(events[:, 2])
    assert np.array_equal(count, want), count
    assert events.shape == (100, 3), events.shape
    if design == 'block':
        events = events[0::5]
        duration = 20.
        assert np.array_equal(np.bincount(events[:, 2]), [0] + [5] * 4)
    else:
        assert design == 'event'
        assert len(events) == 100
        duration = 1.8
        assert events.shape == (100, 3)
        events_r = events[:, 2].reshape(20, 5)
        assert (events_r == events_r[:, :1]).all()
        del events_r
    idx = (events[:, [0, 2]] - [0, 1]).T
    assert np.in1d(idx[1], np.arange(len(conditions))).all()
    stim[tuple(idx)] = 1
#    assert raw_h.info['sfreq'] == sfreq  # necessary for below logic to work
    n_block = int(np.ceil(duration * sfreq))
    stim = signal.fftconvolve(stim, np.ones((n_block, 1)), axes=0)[:n_times]
    dm_events = pd.DataFrame({
        'trial_type': [conditions[ii] for ii in idx[1]],
        'onset': idx[0] / raw_h.info['sfreq'],
        'duration': n_block / raw_h.info['sfreq']})
    dm = make_first_level_design_matrix(
        raw_h.times, dm_events, hrf_model='glover',
        drift_model='polynomial', drift_order=0)
    return stim, dm, events

#mne.viz.plot_events(events)

###############################################################################
# Plot the design matrix and some raw traces

#fig, axes = plt.subplots(2, 1, figsize=(6., 3), constrained_layout=True)
# Design
#ax = axes[0]
#raw_h = use
#stim, dm, _ = _make_design(raw_h, design)

# for ci, condition in enumerate(conditions):
#     color = condition_colors[condition]
#     ax.fill_between(
#         raw_h.times, stim[:, ci], 0, edgecolor='none', facecolor='k',
#         alpha=0.5)
#     model = dm[conditions[ci]].to_numpy()
#     ax.plot(raw_h.times, model, ls='-', lw=1, color=color)
#     x = raw_h.times[np.where(model > 0)[0][0]]
#     ax.text(
#         x + 10, 1.1, condition, color=color, fontweight='bold', ha='center')
# ax.set(ylabel='Modeled\noxyHb', xlabel='', xlim=raw_h.times[[0, -1]])

# # HbO/HbR
# ax = axes[1]
# picks = [pi for pi, ch_name in enumerate(raw_h.ch_names)
#          if 'S1_D2' in ch_name]
# assert len(picks) == 2
# fnirs_colors = dict(hbo='r', hbr='b')
# #ylim = np.array([-1, 1])
# for pi, pick in enumerate(picks):
#     color = fnirs_colors[raw_h.ch_names[pick][-3:]]
#     data = raw_h.get_data(pick)[0] * 1e6
#     val = np.ptp(data)
#     assert val > 0.01
#     ax.plot(raw_h.times, data, color=color, lw=1.)
# ax.set(xlabel='Time (s)', ylabel='μM',
# #       ylim=ylim, 
#        xlim=raw_h.times[[0, -1]])
# #del raw_h
# for ax in axes:
#     for key in ('top', 'right'):
#         ax.spines[key].set_visible(False)
# for ext in ('png', 'svg'):
#     fig.savefig(
#         op.join(
#             results_path, f'figure_1_{exp_name}.{ext}'))

print("All done!") 


In [ ]:
###############################################################################
# Run GLM analysis and epoching

#### CAN SKIP! ####

sfreq = 4.807692050933838

df_cha_list = []
for day in days:
    for subject in subjects:
        group = subject_to_group.get(int(subject), "unknown")
        subj_cha_list = []
        t0 = time.time()
        for run in runs:
            print(f'Running GLM for {group} {subject} day {day} run {run:03d}... ', end='')
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname2)
            _, dm, _ = _make_design(raw_h, design, subject, run, day, group)
            glm_est = mne_nirs.statistics.run_glm(
                raw_h, dm, noise_model='ols', n_jobs=n_jobs)
            cha = glm_est.to_dataframe()
            cha['subject'] = subject
            cha['run'] = run
            cha['day'] = day
            cha['group'] = group
            subj_cha_list.append(cha)
            del raw_h
        subj_cha = pd.concat(subj_cha_list, ignore_index=True)
        df_cha_list.append(subj_cha) # test
        print(f'{time.time() - t0:0.1f} sec') # test
        
#            print(f'Writing HDF5 for subject {subject}, day {day}... ', end='')
#            subj_cha.to_hdf(fname, key='subj_cha', mode='w')
#            print(f'{time.time() - t0:0.1f} sec')
#        df_cha_list.append(pd.read_hdf(fname))

# Test
df_cha = pd.concat(df_cha_list, ignore_index=True)
df_cha.reset_index(drop=True, inplace=True)
df_cha.to_csv(op.join(results_path, 'df_cha.csv'), index=False)

print("All done!") 

In [ ]:
# To skip above block, just run this:

df_cha = pd.read_csv(op.join(results_path, 'df_cha.csv'))

In [ ]:
# block averages
event_id = {condition: ci for ci, condition in enumerate(conditions, 1)}
evokeds = {condition: dict() for condition in conditions}
for day in days:
    for subject in subjects:
        fname = op.join(proc_path, f'{subject}_{day}_{exp_name}-ave.fif')
        if not op.isfile(fname):
            tmin, tmax = -2, 38
            baseline = (None, 0)
            t0 = time.time()
            print(f'Creating block average for {subject} day {day}... ', end='')
            raws = list()
            events = list()
            for run in runs:
                fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
                raw_h = mne.io.read_raw_fif(fname2)
                events.append(_make_design(raw_h, 'block', subject, run)[2])
                raws.append(raw_h)
            bads = sorted(set(sum((r.info['bads'] for r in raws), [])))
            for r in raws:
                r.info['bads'] = bads
            raw_h, events = mne.concatenate_raws(raws, events_list=events)
            epochs = mne.Epochs(raw_h, events, event_id, tmin=tmin, tmax=tmax,
                                baseline=baseline)
            this_ev = [epochs[condition].average() for condition in conditions]
            assert all(ev.nave > 0 for ev in this_ev)
            mne.write_evokeds(fname, this_ev, overwrite=True)
            print(f'{time.time() - t0:0.1f} sec')
            for condition in conditions:
                evokeds[condition][subject] = mne.read_evokeds(fname, condition)
            print(f'Done for {group} {subject} day {day} run {run:03d}... ', end='')

print("All done!") 


In [ ]:
# Make life easier by combining across runs

# Mark bad channels 
bad = dict()
bb = dict()

for day in days:
    for subject in subjects:
        for run in runs:
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            this_info = mne.io.read_info(fname2)
            bad_channels = [idx - 1 for idx in sorted(
                this_info['ch_names'].index(bad) + 1 for bad in this_info['bads'])]
            valid_indices = np.arange(len(use.ch_names))
            bb = [b for b in bad_channels if b in valid_indices]
            bad[(subject, run, day)] = bb
        assert np.in1d(bad[(subject, run, day)], np.arange(len(use.ch_names))).all()  # noqa: E501

bad_combo = dict()
for day in days:
    for (subject, run, day), bb in bad_channels:
        bad_combo[subject] = sorted(set(bad_combo.get(subject, [])) | set(bb))
bad = bad_combo
#assert set(bad) == set(subjects)

start = len(df_cha)
n_drop = 0
for day in days:
    for (subject, run, day), bb in bad_channels:
        if not len(bb):
            continue
        drop_names = [use.ch_names[b] for b in bb]
        is_subject = (df_cha['subject'] == subject)
        is_day = (df_cha['day'] == day)
        assert len(is_subject) == len(df_cha)
        is_day = (df_cha['day'] == day)
        drop = df_cha.index[
            is_subject &
            is_day &
            np.in1d(df_cha['ch_name'], drop_names)]
        n_drop += len(drop)
        if len(drop):
            print(f'Dropping {len(drop)} for {subject} day {day}')  # {run}')
            df_cha.drop(drop, inplace=True)
end = len(df_cha)
assert n_drop == start - end, (n_drop, start - end)

# combine runs by averaging 
sorts = ['subject', 'ch_name', 'Chroma', 'Condition', 'group', 'day', 'run']
df_cha.sort_values(
    sorts, inplace=True)
#assert (np.array(df_cha['run']).reshape(-1, 2) == runs).all()
theta = np.array(df_cha['theta']).reshape(-1, len(runs)).mean(-1)
df_cha.drop(
    [col for col in df_cha.columns if col not in sorts[:-1]], axis='columns',
    inplace=True)
df_cha.reset_index(drop=True, inplace=True)
df_cha = df_cha[::len(runs)]
df_cha.reset_index(drop=True, inplace=True)
df_cha['theta'] = theta
df_cha.to_csv(op.join(results_path, 'df_cha_theta.csv'), index=False)

print("All done!") 


In [ ]:
# Mixed linear model
def _mixed_df(ch_summary):
    formula = "theta ~ -1 + ch_name:Condition" 
    ch_model = smf.mixedlm(
        formula, ch_summary, groups=ch_summary["subject"]).fit(method='powell')
    ch_model_df = mne_nirs.statistics.statsmodels_to_results(ch_model)
    ch_model_df['P>|z|'] = ch_model.pvalues
    ch_model_df.drop([idx for idx in ch_model_df.index if '[constant]' in idx],
                    inplace=True)
    return ch_model_df

# Make separate subject lists for trained and untrained (TEST)
#trained_subjects = {'201 203 207 212 213 214 216 219 221 223'}
#control_subjects = {'202 204 205 206 208 209 215 217 218 224'}

# Run group level model and convert to dataframe
ch_summary = df_cha.query("Chroma in ['hbo']").copy()
#ch_summary = df_cha.query("Chroma in ['hbo'] and group in ['trained'] and day in ['3']").copy()
ch_model_df = _mixed_df(ch_summary) 
ch_model_df.reset_index(inplace=True)

# Correct for multiple comparisons
print(f'Correcting for {len(ch_model_df["P>|z|"])} comparisons using FDR')
_, ch_model_df['P_fdr'] = mne.stats.fdr_correction(
    ch_model_df['P>|z|'], method='indep')
ch_model_df['SIG'] = ch_model_df['P_fdr'] < 0.05
ch_model_df.to_csv(op.join(results_path, 'ch_model_corrected_hbo.csv'), index=False)
ch_model_df.loc[ch_model_df.SIG == True]

print("All done!") 

In [ ]:
# SKIP!!!
# # Analyze HbR Data Separately

# ch_summary_hbr = df_cha.query("Chroma in ['hbr']").copy()
# ch_summary_use_hbr = ch_summary_hbr.query(
#     f"subject in {use_subjects}").copy()
# ch_model_df_hbr = _mixed_df(ch_summary_use_hbr) 
# ch_model_df_hbr.reset_index(inplace=True)

# # Correct for multiple comparisons
# print(f'Correcting for {len(ch_model_df_hbr["P>|z|"])} comparisons using FDR')
# _, ch_model_df_hbr['P_fdr'] = mne.stats.fdr_correction(
#     ch_model_df_hbr['P>|z|'], method='indep')
# ch_model_df_hbr['SIG'] = ch_model_df_hbr['P_fdr'] < 0.05
# ch_model_df_hbr.to_csv(op.join('ch_model_corrected_hbr.csv'), index=False)
# ch_model_df_hbr.to_csv(op.join(results_path, 'ch_model_corrected_hbr.csv'), index=False)
# ch_model_df_hbr.loc[ch_model_df_hbr.SIG == True]

# print("All done!") 


In [ ]:
###############################################################################
# Plot significant channels
sig_chs = dict()
zs = dict()
for condition in conditions:
    sig_df = ch_model_df[
        (ch_model_df['P_fdr'] < 0.05) &
        (ch_model_df['Condition'] == condition) &
        (ch_model_df['ch_name'].isin(use.ch_names))
        ]
    sig_chs[(condition)] = sorted(
        (use.ch_names.index(row[1]['ch_name']), row[1]['P_fdr'])
        for row in sig_df.iterrows())
    ch_model_df[ch_model_df['Condition'] == condition]
    zs[condition] = np.array([
        ch_model_df.loc[(ch_model_df['Condition'] == condition) & 
                        (ch_model_df['ch_name'] == ch_name), 'z'].iloc[0]
        for ch_name in info['ch_names'][::2]], float)
    assert zs[condition].shape == (84,)
    assert np.isfinite(zs[condition]).all()

def _plot_sig_chs(sigs, ax):
    if sigs and isinstance(sigs[0], tuple):
        sigs = [s[0] for s in sigs]
    ch_groups = [sigs, np.setdiff1d(np.arange(info['nchan']), sigs)]
    mne.viz.plot_sensors(
        info, 'topomap', 'hbo', title='', axes=ax,
        show_names=True, ch_groups=ch_groups)
    ax.collections[0].set(lw=0)
    c = ax.collections[0].get_facecolor()
    c[(c[:, :3] == (0.5, 0, 0)).all(-1)] = (0., 0., 0., 0.1)
    c[(c[:, :3] == (0, 0, 0.5)).all(-1)] = (0., 1., 0., 0.5)
    ax.collections[0].set_facecolor(c)
    ch_names = [info['ch_names'][idx] for idx in sigs]
    texts = list(ax.texts)
    got = []
    for text in list(texts):
        try:
            idx = ch_names.index(text.get_text())
        except ValueError:
            text.remove()
        else:
            got.append(idx)
            text.set_text(f'{sigs[idx] // 2 + 1}')
            text.set(fontsize='xx-small', zorder=5, ha='center')
    assert len(got) == len(sigs), (got, list(sigs))

def _plot_sigs(sig_chs, all_corrs=()):
    n_col = max(len(x) for x in sig_chs.values()) + 1
    n_row = len(conditions)
    figsize = (n_col * 1.0, n_row * 1.0)
    fig, axes = plt.subplots(
        n_row, n_col, figsize=figsize, constrained_layout=True, squeeze=False)
    h_colors = {0: 'r', 1: 'b'}
    xticks = [0, 10, 20, 30]
    ylim = [-0.2, 0.3]
    yticks = [-0.2, -0.1, 0, 0.1, 0.2, 0.3]
    xlim = [times[0], 35]
    ylim = np.array(ylim)
    yticks = np.array(yticks)
    for ci, condition in enumerate(conditions):
        ii = 0
        sigs = sig_chs[condition]
        if len(sigs) == 0:
            sigs = [(None, None)]
        for ii, (ch_idx, ch_p) in enumerate(sigs):
            ax = axes[ci, ii]
            if ch_idx is not None:
                for jj in range(2):  # HbO, HbR
                    color = h_colors[jj]
                    a = 1e6 * np.array(
                        [evokeds[condition][subject].data[ch_idx + jj]
                         for subject in use_subjects
                         if ch_idx + jj not in bad.get(subject, [])], float)
                    m = np.mean(a, axis=0)
                    lower, upper = stats.t.interval(
                        0.95, len(a) - 1, loc=m, scale=stats.sem(a, axis=0))
                    ax.fill_between(
                        times, lower, upper, facecolor=color,
                        edgecolor='none', lw=0, alpha=0.25, zorder=3,
                        clip_on=False)
                    ax.plot(times, m, color=color, lw=1, zorder=4,
                            clip_on=False)
                # Correlations
                this_df = ch_summary_use.query(
                    f'ch_name == {repr(use.ch_names[ch_idx])} and '
                    f'Chroma == "hbo" and '
                    f'Condition == {repr(condition)}')
                #assert 8 <= len(this_df) <= len(subjects), len(this_df)
                a = np.array(this_df['theta'])
                cs = list()
                if len(cs):
                    cs = [''] + cs
                c = '\n'.join(cs)
                ax.text(times[-1], ylim[1],
                        f'ch{ch_idx // 2 + 1}\np={ch_p:0.5f}{c}',
                        ha='right', va='top', fontsize='x-small')
            ax.axvline(20, ls=':', color='0.5', zorder=2, lw=1)
            ax.axhline(0, ls='-', color='k', zorder=2, lw=0.5)
            ax.set(xticks=xticks, yticks=yticks)
            ax.set(xlim=xlim, ylim=ylim)
            for key in ('top', 'right'):
                ax.spines[key].set_visible(False)
            if ax.get_subplotspec().is_last_row():
                ax.set(xlabel='Time (sec)')
            else:
                ax.set_xticklabels([''] * len(xticks))
            if ax.get_subplotspec().is_first_col():
                ax.set_ylabel(condition)
            else:
                ax.set_yticklabels([''] * len(yticks))
            for key in ('top', 'right'):
                ax.spines[key].set_visible(False)
        for ii in range(ii + 1, n_col - 1):
            fig.delaxes(axes[ci, ii])
        # montage
        ax = axes[ci, -1]
        if sigs[0][0] is None:
            fig.delaxes(ax)
        else:
            # plot montage
            _plot_sig_chs(sigs, ax)
    return fig

times = evokeds[conditions[0]][subjects[0]].times
info = evokeds[conditions[0]][subjects[0]].info
fig = _plot_sigs(sig_chs)
#for ext in ('png', 'svg'):
#    fig.savefig(op.join(results_path, f'stats_{exp_name}.{ext}'))

print("All done!") 


In [ ]:
# ONLY PLOT SENSORS
def _plot_sigs(sig_chs, all_corrs=()):
    n_col = 1  # Only need one column for sensor locations
    n_row = len(conditions)
    figsize = (n_col * 2, n_row * 2)  # Increase figure size for better label visibility
    fig, axes = plt.subplots(
        n_row, n_col, figsize=figsize, constrained_layout=True)

    # Handle the case of a single subplot
    if n_row == 1:
        axes = [axes]

    for ci, condition in enumerate(conditions):
        sigs = sig_chs[condition]
        ax = axes[ci]  # Direct reference to each subplot's axis
        
        # Ensure labels are set even if there are no significant sensors
        ax.set_ylabel(condition, labelpad=100)  # Increase labelpad if necessary

        # Only attempt to plot sensor locations if there are significant sensors
        if sigs[0][0] is not None:
            _plot_sig_chs(sigs, ax)
        else:
            # Optionally, clear the axes but keep them to display the label
            ax.clear()  # Clear the axes of any plotted data
            ax.set_xticks([])  # Remove x-ticks
            ax.set_yticks([])  # Remove y-ticks
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
    return fig

times = evokeds[conditions[0]][subjects[0]].times
info = evokeds[conditions[0]][subjects[0]].info
fig = _plot_sigs(sig_chs)

print("All done!") 

In [ ]:
###############################################################################
# Source space projection

info = use.copy().pick_types(fnirs='hbo', exclude=()).info
info['bads'] = []
assert tuple(zs) == conditions

evoked = mne.EvokedArray(np.array(list(zs.values())).T, info)
picks = np.arange(len(evoked.ch_names))

for ch in evoked.info['chs']:
    assert ch['coord_frame'] == mne.io.constants.FIFF.FIFFV_COORD_HEAD
stc = mne.stc_near_sensors(
    evoked, trans='fsaverage', subject='fsaverage', mode='weighted',
    distance=0.02, project=True, picks=picks, subjects_dir=subjects_dir)
# Split channel indices by left lat, posterior, right lat:
num_map = {name: str(ii) for ii, name in enumerate(evoked.ch_names)}
evoked.copy().rename_channels(num_map) #.plot_sensors(show_names=True)
view_map = [np.arange(19), np.arange(19, 33), np.arange(33, 52)]
surf = mne.read_bem_surfaces(  # brain surface
    f'{subjects_dir}/fsaverage/bem/fsaverage-5120-5120-5120-bem.fif', s_id=1)

for ci, condition in enumerate(conditions):
    this_sig = [v[0] // 2 for v in sig_chs[condition]]
    #assert np.in1d(this_sig, np.arange(52)).all()
    pos = np.array([info['chs'][idx]['loc'][:3] for idx in this_sig])
    pos.shape = (-1, 3)  # can be empty
    trans = mne.transforms._get_trans('fsaverage', 'head', 'mri')[0]
    pos = mne.transforms.apply_trans(trans, pos)  # now in MRI coords
    pos = mne.surface._project_onto_surface(pos, surf, project_rrs=True)[2]
    # plot
    brain = stc.plot(hemi='both', views=['lat', 'frontal', 'lat'],
                    initial_time=evoked.times[ci], cortex='low_contrast',
                    time_viewer=False, show_traces=False,
                    surface='pial', smoothing_steps=0, size=(1200, 400),
                    clim=dict(kind='value', pos_lims=[0., 1.25, 2.5]),
                    colormap='RdBu_r', view_layout='horizontal',
                    colorbar=(0, 1), time_label='', background='w',
                    brain_kwargs=dict(units='m'),
                    add_data_kwargs=dict(colorbar_kwargs=dict(
                        title_font_size=24, label_font_size=24, n_labels=5,
                        title='z score')), subjects_dir=subjects_dir)
    brain.show_view('lat', hemi='lh', row=0, col=0)
    brain.show_view(azimuth=270, elevation=90, row=0, col=1)
    pl = brain.plotter
    used = np.zeros(len(this_sig))
    brain.show_view('lat', hemi='rh', row=0, col=2)
    plt.imsave(
        op.join(results_path, f'all_brain_{exp_name}_{condition}.png'), pl.image)

print("All done!") 

In [ ]:
# fOLD specificity
fold_files = ['10-10.xls', '10-5.xls']
for fname in fold_files:
    if not op.isfile(fname):
        pooch.retrieve(f'https://github.com/nirx/fOLD-public/raw/master/Supplementary/{fname}', None, fname, path=os.getcwd())  # noqa
raw_spec = use['h'].copy()
raw_spec.pick_channels(raw_spec.ch_names[::2])
specs = mne_nirs.io.fold_channel_specificity(raw_spec, fold_files, 'Brodmann')
for si, spec in enumerate(specs, 1):
    spec['Channel'] = si
    spec['negspec'] = -spec['Specificity']
specs = pd.concat(specs, ignore_index=True)
specs.drop(['Source', 'Detector', 'Distance (mm)', 'brainSens',
            'X (mm)', 'Y (mm)', 'Z (mm)'], axis=1, inplace=True)
specs.sort_values(['Channel', 'negspec'], inplace=True)
specs.drop('negspec', axis=1, inplace=True)
specs.reset_index(inplace=True, drop=True)
specs.to_csv(op.join(results_path, 'specificity.csv'), index=False)

In [4]:
# HbDiff Analysis

df_cha = pd.read_csv(op.join(results_path, 'df_cha_theta.csv'))
df_cha['ch_name'] = df_cha['ch_name'].str[:-4]

# run twice for trained vs. control
df_hbo = df_cha[df_cha['Chroma'].str.endswith('hbo')].set_index(['subject', 'Condition', 'group', 'day', 'ch_name']).sort_index()
df_hbr = df_cha[df_cha['Chroma'].str.endswith('hbr')].set_index(['subject', 'Condition', 'group', 'day', 'ch_name']).sort_index()

# Compute the difference
df_cha_diff_list = []
for ch_name in df_cha['ch_name'].unique():
    df_cha_ch = df_cha[df_cha['ch_name'] == ch_name].copy()
    if df_cha_ch.duplicated(subset=['subject', 'Condition', 'group', 'day']).any():
        df_cha_ch.drop_duplicates(subset=['subject', 'Condition', 'group', 'day'], inplace=True)
    df_diff = df_hbo[df_hbo.index.get_level_values('ch_name') == ch_name][['theta']].reset_index(drop=True).sub(
        df_hbr[df_hbr.index.get_level_values('ch_name') == ch_name][['theta']].reset_index(drop=True)
    )
    df_cha_diff = df_cha_ch.reset_index(drop=True).copy()
    df_cha_diff['theta'] = df_diff.values
    if not df_cha_diff.empty:
        df_cha_diff_list.append(df_cha_diff)

df_cha_diff_concat = pd.concat(df_cha_diff_list, ignore_index=True)
df_cha_diff_concat.to_csv(op.join(results_path, 'df_cha_HbDiff.csv'), index=False)  ###
#print(df_cha_diff_concat)
#print("All done!")


In [6]:
import pandas as pd

df_cha = pd.read_csv(op.join(results_path, 'df_cha_HbDiff.csv'))

# run twice for trained vs. control
ch_model_df_diff = pd.DataFrame()
#df_day1 = df_cha.query("Chroma in ['hbo'] and group == ['control'] and day in [1]").copy()  ###
#df_day3 = df_cha.query("Chroma in ['hbo'] and group == ['control'] and day in [3]").copy()  ###
df_day1 = df_cha.query("group == ['trained'] and day in [1]").copy()  ###
df_day3 = df_cha.query("group == ['trained'] and day in [3]").copy()  ###

df_day1 = df_day1.set_index(['subject', 'ch_name', 'Condition', 'Chroma']).sort_index()
df_day3 = df_day3.set_index(['subject', 'ch_name', 'Condition', 'Chroma']).sort_index()

# Compute the difference
df_diff = df_day3[['theta']] - df_day1[['theta']]
df_diff = df_diff.reset_index()
ch_model_df_diff = df_diff.copy()

def _mixed_df(ch_summary):
    formula = "theta ~ -1 + ch_name:Condition" 
    ch_model = smf.mixedlm(
        formula, ch_summary, groups=ch_summary["subject"]).fit(method='powell')
    ch_model_df = mne_nirs.statistics.statsmodels_to_results(ch_model)
    ch_model_df['P>|z|'] = ch_model.pvalues
    ch_model_df.drop([idx for idx in ch_model_df.index if '[constant]' in idx],
                    inplace=True)
    return ch_model_df

# Correct for multiple comparisons
ch_model_df_diff = _mixed_df(ch_model_df_diff) 
print(f'Correcting for {len(ch_model_df_diff["P>|z|"])} comparisons using FDR')
_, ch_model_df_diff['P_fdr'] = mne.stats.fdr_correction(
    ch_model_df_diff['P>|z|'], method='indep')
ch_model_df_diff['SIG'] = ch_model_df_diff['P_fdr'] < 0.05
ch_model_df_diff.to_csv(op.join(results_path, 'ch_model_trained_diff_hbo.csv'), index=False)  ###
ch_model_df_diff.loc[ch_model_df_diff.SIG == True]
print(ch_model_df_diff)

# Use the ch_model_df_diff for significant channels and z-scores
ch_model_df = ch_model_df_diff.copy()
ch_model_df = ch_model_df.loc[ch_model_df.SIG == True]
#print(ch_model_df)

print("All done!")


/Users/ansle/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Correcting for 368 comparisons using FDR
                                      Coef. Std.Err.         z     P>|z|  \
ch_name[S10_D18]:Condition[A]  1.789201e-08      0.0  0.127024  0.898921   
ch_name[S10_D20]:Condition[A]  1.385316e-07      0.0  0.983506  0.325359   
ch_name[S10_D22]:Condition[A]  1.574374e-07      0.0  1.117728  0.263683   
ch_name[S10_D30]:Condition[A] -8.827109e-08      0.0 -0.407850  0.683384   
ch_name[S11_D19]:Condition[A]  2.442712e-07      0.0  1.734205  0.082882   
...                                     ...      ...       ...       ...   
ch_name[S8_D20]:Condition[W]   1.252584e-07      0.0  0.889273  0.373856   
ch_name[S9_D17]:Condition[W]  -7.341379e-09      0.0 -0.052120  0.958433   
ch_name[S9_D19]:Condition[W]  -3.433277e-08      0.0 -0.243746  0.807428   
ch_name[S9_D20]:Condition[W]  -5.735793e-08      0.0 -0.407213  0.683852   
ch_name[S9_D21]:Condition[W]  -8.960140e-08      0.0 -0.636126  0.524694   

                                 [0.025    0.9

In [7]:
# Source space projection - differences
ch_model_df = pd.read_csv(op.join(results_path, 'ch_model_trained_diff_hbo.csv'))

fname = op.join(proc_path, f'{plot_subject}_{plot_day}_001_hbo_raw.fif')
use = mne.io.read_raw_fif(fname)
info = use.info

ch_of_interest = use.pick_channels([ch_name for ch_name in info['ch_names'] if ch_name.endswith('hbo')])
ch_of_interest.rename_channels(lambda x: x[:-4] if x.endswith(' hbo') else x) ####
info_of_interest = ch_of_interest.info

zs = {}
for condition in conditions:
    condition_data = ch_model_df[ch_model_df['Condition'] == condition]
#    zs[condition] = np.array([condition_data.loc[condition_data['ch_name'] == ch_name, 'z'].values[0] for ch_name in info_of_interest['ch_names']]) ###
    zs[condition] = np.array([condition_data.loc[(condition_data['Condition'] == condition) & (condition_data['ch_name'] == ch_name), 'z'].iloc[0] for ch_name in info_of_interest['ch_names']], float)

# Create an EvokedArray
evoked = mne.EvokedArray(np.array(list(zs.values())).T, info_of_interest)
picks = np.arange(len(info_of_interest['ch_names']))

#info = use.copy().pick_types(fnirs='hbo', exclude=()).info ###
#info['bads'] = []
assert tuple(zs) == conditions

stc = mne.stc_near_sensors(
    evoked, trans='fsaverage', subject='fsaverage', mode='weighted',
    distance=0.02, project=True, picks=picks, subjects_dir=subjects_dir)

# Split channel indices by left lat, posterior, right lat:
num_map = {name: str(ii) for ii, name in enumerate(info_of_interest['ch_names'])}
evoked.copy().rename_channels(num_map) #.plot_sensors(show_names=True)
view_map = [np.arange(19), np.arange(19, 33), np.arange(33, 52)]
surf = mne.read_bem_surfaces(  # brain surface
    f'{subjects_dir}/fsaverage/bem/fsaverage-5120-5120-5120-bem.fif', s_id=1)

for ci, condition in enumerate(conditions):
    trans = mne.transforms._get_trans('fsaverage', 'head', 'mri')[0]
    brain = stc.plot(hemi='both', views=['lat', 'frontal', 'lat'],
                    initial_time=evoked.times[ci], cortex='low_contrast',
                    time_viewer=False, show_traces=False,
                    surface='pial', smoothing_steps=0, size=(1200, 400),
                    clim=dict(kind='value', pos_lims=[0, 1.5, 3]),
                    colormap='RdBu_r', view_layout='horizontal',
                    colorbar=(0, 1), time_label='', background='w',
                    brain_kwargs=dict(units='m'),
                    add_data_kwargs=dict(colorbar_kwargs=dict(
                        title_font_size=14, label_font_size=12, n_labels=5,
                        title='z score')), subjects_dir=subjects_dir)
    brain.show_view('lat', hemi='lh', row=0, col=0)
    brain.show_view(azimuth=270, elevation=90, row=0, col=1)
    pl = brain.plotter
    brain.show_view('lat', hemi='rh', row=0, col=2)
    plt.imsave(
        op.join(results_path, f'trained_HbDiff_diff_{condition}.png'), pl.image)  ###

print("All done!") 


Opening raw data file ../../processed/201_1_001_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Projecting data from 84 sensors onto 327684 surface vertices: weighted mode
    Projecting sensors onto surface
    Minimum projected intra-sensor distance: 6.1 mm
    160208 / 327684 non-zero vertices
    3 BEM surfaces found
Using pyvistaqt 3d backend.



/Users/ansle/opt/anaconda3/lib/python3.9/site-packages/pyvista/plotting/plotting/__init__.py:23: PyVistaDeprecationWarning: The `pyvista.plotting.plotting` module has been deprecated. `_ALL_PLOTTERS` is now imported as: `from pyvista.plotting import _ALL_PLOTTERS`.
  warnings.warn(
2024-07-24 18:56:56.358 python[93290:5416551] NSKeyBindingManager: Bad key binding selector for '^z' = 'undo:'


All done!
